## CBOE SKEW analysis

In [1]:
import sys,os

if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))
sys.path.append(os.path.abspath('../../dashgrid'))
sys.path.append(os.path.abspath('../../dashgrid/dashgrid'))
# print(sys.path)
from dashgrid import dgrid
import dash_core_components as dcc
import traceback

from volgrid import create_voltables
import plotly.graph_objs as go
from plotly.offline import iplot
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import numpy as np
import pandas as pd
import pandas_datareader.data as pdr
import datetime
import pytz

#  do rest of imports
import dash
import dash_html_components as html
from dash.dependencies import Input, Output,State
import yfinance as yf
import pathlib

### Paremeters to calculate SKEW

In [2]:
SKEW_TICKER = '^SPX' # yahoo symbol
THRESHOLD_BID = .1 # minimum bid value when screening options
DELTA_K = 5 # strike difference between most of the options that get used to calculate SKEW
TIMEZONE = 'US/Eastern'
TRADE_DATE = 20191224
EXP_NEAR_YYYYMMDD = 20200116
EXP_FAR_YYYYMMDD = 20200220
INTEREST_RATE = None # calculate at run time ( this will be the average of the 1 month and 2 month libor rates from FRED)

OUTPUT_COLS = ['contractSymbol','strike','mid','dte_pct','ert',
                'forward_price','deltak','p1','p2','p3','e1','e2','e3']


In [3]:
df_skew = pd.read_csv('https://www.cboe.com/publish/scheduledtask/mktdata/datahouse/skewdailyprices.csv',header=1)

In [4]:
df_skew

,Date,SKEW,Unnamed: 2,Unnamed: 3
0,1/2/1990,126.09,NaN,NaN
1,1/3/1990,123.34,NaN,NaN
2,1/4/1990,122.62,NaN,NaN
3,1/5/1990,121.27,NaN,NaN
4,1/8/1990,124.12,NaN,NaN
...,...,...,...,...
7539,12/18/2019,144.56,NaN,NaN
7540,12/19/2019,150.14,NaN,NaN
7541,12/20/2019,147.17,NaN,NaN
7542,12/23/2019,143.91,NaN,NaN


In [5]:
spx = yf.Ticker(SKEW_TICKER)
spx.options

('2019-12-26',
 '2019-12-29',
 '2019-12-30',
 '2020-01-02',
 '2020-01-05',
 '2020-01-07',
 '2020-01-09',
 '2020-01-12',
 '2020-01-14',
 '2020-01-16',
 '2020-01-20',
 '2020-01-21',
 '2020-01-23',
 '2020-01-26',
 '2020-01-30',
 '2020-02-06',
 '2020-02-13',
 '2020-02-20',
 '2020-02-27',
 '2020-03-05',
 '2020-03-19',
 '2020-03-30',
 '2020-04-16',
 '2020-04-29',
 '2020-05-14',
 '2020-05-28',
 '2020-06-18',
 '2020-06-29',
 '2020-09-17',
 '2020-09-18',
 '2020-09-29',
 '2020-10-15',
 '2020-11-19',
 '2020-12-17',
 '2021-01-14',
 '2021-03-18',
 '2021-06-17',
 '2021-12-16',
 '2026-03-19')

In [6]:
def dt_from_yyyymmdd(yyyymmdd,hour=0,minute=0,timezone=TIMEZONE):
    y = int(str(yyyymmdd)[0:4])
    m = int(str(yyyymmdd)[4:6])
    d = int(str(yyyymmdd)[6:8])  
    return datetime.datetime(y,m,d,hour,minute,tzinfo=pytz.timezone(timezone))
def get_dte_pct(trade_yyyymmdd,expiry_yyyymmdd):
    dt_td = dt_from_yyyymmdd(trade_yyyymmdd)
    dt_xp = dt_from_yyyymmdd(expiry_yyyymmdd)
    return ((dt_xp - dt_td).days + 1)/365



In [7]:
def get_rate(num_months_for_rate=1):
    # see if you can get Libor from the FRED API
    n = datetime.datetime.now() - datetime.timedelta(14)
    y = n.year
    m = n.month
    d = n.day
    beg = '%04d-%02d-%02d' %(y,m,d)
    df = pdr.DataReader(f'USD{num_months_for_rate}MTD156N', "fred", f'{beg}', '2200-12-31')
    fixed_rate = float(df.iloc[len(df)-1][f'USD{num_months_for_rate}MTD156N'])/100
    return fixed_rate


In [47]:
ed = str(EXP_FAR_YYYYMMDD)
dd = f'{ed[0:4]}-{ed[4:6]}-{ed[6:8]}'
c = yf.Ticker('^SPX')
df_t = c.option_chain(dd)
              

In [49]:
df_tp = df_t.puts[(df_t.puts.contractSymbol.str.slice(0,4)=='SPX2')]
df_tp[df_tp.strike>=3220.0].iloc[:3]

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
228,SPX200221P03220000,2019-12-24 17:32:46,3220.0,55.02,53.9,54.6,1.220001,2.267660,110.0,63,0.108960,False,REGULAR,USD
230,SPX200221P03230000,2019-12-24 16:02:01,3230.0,58.60,57.6,58.2,1.399998,2.447548,1.0,8,0.105996,True,REGULAR,USD
232,SPX200221P03240000,2019-12-20 20:34:13,3240.0,62.30,61.6,62.2,0.000000,0.000000,12.0,45,0.103238,True,REGULAR,USD


In [40]:
df_tc = df_t.calls[(df_t.calls.bid>.1) & (df_t.calls.contractSymbol.str.slice(0,4)=='SPX2')]
df_tc[~df_tc.inTheMoney].iloc[:3]

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
173,SPX200117C03225000,2019-12-24 18:13:56,3225.0,31.15,30.9,31.5,-0.850000,-2.656251,3877.0,16419,0.097909,False,REGULAR,USD
174,SPX200117C03230000,2019-12-24 17:59:39,3230.0,27.72,27.9,28.5,-0.370001,-1.317198,577.0,8576,0.096010,False,REGULAR,USD
175,SPX200117C03235000,2019-12-24 17:58:54,3235.0,24.50,25.0,25.6,-1.200001,-4.669263,121.0,4142,0.094011,False,REGULAR,USD


In [85]:
import pdb
def get_valid_series_from_yfinance(trade_date_yyyymmdd,expiry_yyyymmdd,ticker=SKEW_TICKER,threshold_bid=THRESHOLD_BID,interest_rate=None,deltak=DELTA_K):
    # ****** Step 01: create interest rate to use if necessary
    ir = interest_rate
    if ir is None:
        ir  = (get_rate(1) + get_rate(2))/2
    
    # ******  Step 02:  get yfinance contract, and options series
    contract = yf.Ticker(ticker)
    # reformat expiry_yyyymmdd into something like 2019-12-20
    ed = str(expiry_yyyymmdd)
    date_string = f'{ed[0:4]}-{ed[4:6]}-{ed[6:8]}'
    # get the options chain from yahoo finance
    df_opt = contract.option_chain(date_string)
    
    # ****** Step 03: figure out forward price
    df_spx_c = df_opt.calls[(df_opt.calls.bid>threshold_bid) & (df_opt.calls.contractSymbol.str.slice(0,4)=='SPX2')]
    df_spx_c['cp'] = 'c'
    df_spx_p = df_opt.puts[(df_opt.puts.bid>threshold_bid) & (df_opt.puts.contractSymbol.str.slice(0,4)=='SPX2')]
    df_spx_p['cp'] = 'p'
    
    # ******  Step 03: extract out the ATM and the out of the money calls
    df_spx_c = df_spx_c[~df_spx_c.inTheMoney]
    df_spx_c = df_spx_c.sort_values('strike')
    df_spx_c.index = range(len(df_spx_c))
    
    first_call_strike = df_spx_c.iloc[0].strike
    # save the lowest_itm_put for later calc of forward price
    call_strikes = df_spx_c.strike.values
    lowest_itm_put = df_spx_p[(df_spx_p.strike>=first_call_strike) & df_spx_p.strike.isin(call_strikes)].iloc[0]
    
    
    # ******  Step 04:  extract out of the money puts
    #    get atm option strike (which is first call)
    df_spx_p = df_spx_p[~df_spx_p.inTheMoney]
    df_spx_p = df_spx_p.sort_values('strike')
    df_spx_p.index = range(len(df_spx_p))

    
    # ******  Step 05: merge puts and alls into one dataframe
    df_ret = df_spx_p.copy()
    df_ret = df_ret.append(df_spx_c)
    df_ret = df_ret.sort_values(['cp','strike'])
    df_ret.index = range(len(df_ret))
    df_ret['expiry_yyyymmdd'] = expiry_yyyymmdd
    df_ret['trade_date_yyyymmdd'] = trade_date_yyyymmdd

    # ******  Step 06: Add mid price, expiry, dte_pct, ert, forward price     
    df_ret['mid'] = (df_ret.bid + df_ret.ask)/2
    dte_pct = get_dte_pct(trade_date_yyyymmdd,expiry_yyyymmdd)
    df_ret['dte_pct'] = dte_pct
    ert = np.exp(dte_pct * ir)
    df_ret['ert'] = ert
    # find forward price by using the lowest strike call in df_ret 
    #     (what the SKEW whitepaper calls the At The Money option)
    # get ATM call midpoint
    atm_strike = lowest_itm_put.strike 
    print(atm_strike)
    atm_call = df_ret[(df_ret.strike==atm_strike) & (df_ret.cp=='c')].iloc[0]
    atm_call_price = (atm_call.bid + atm_call.ask) / 2 
    atm_put = lowest_itm_put
    atm_put_price = (atm_put.bid + atm_put.ask) / 2 
    forward_price = ert*(atm_call_price - atm_put_price) + atm_strike
    df_ret['forward_price'] = forward_price
    df_ret['k_over_fp'] = df_ret.strike / forward_price
    df_ret['deltak'] = deltak
    # create p1 unit values
    df_ret['p1']= df_ret.apply(lambda r: r.deltak / r.strike**2 * r.mid,axis=1)
    df_ret['p2'] = df_ret.apply(lambda r: 2 * r.p1 *(1-np.log(r.k_over_fp)),axis=1)
    df_ret['p3'] = df_ret.apply(lambda r: 3 * r.p1 * (2*np.log(r.k_over_fp) - np.log(r.k_over_fp)**2),axis=1)
    e1 = -(1+np.log(forward_price/atm_strike) - forward_price/atm_strike)
    e2 = 2 * np.log(atm_strike/forward_price) * (forward_price/atm_strike - 1) + 1/2 * np.log(atm_strike/forward_price)**2
    e3 = 3 * np.log(atm_strike/forward_price)**2 * (1/3 * np.log(atm_strike/forward_price) - 1 + forward_price/atm_strike)
    df_ret['e1'] = e1
    df_ret['e2'] = e2
    df_ret['e3'] = e3
    return df_ret
    

In [89]:
df_series_near = get_valid_series_from_yfinance(TRADE_DATE,EXP_NEAR_YYYYMMDD)
df_series_far = get_valid_series_from_yfinance(TRADE_DATE,EXP_FAR_YYYYMMDD)


/Users/bperlman1/Virtualenvs3/dashrisk3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/bperlman1/Virtualenvs3/dashrisk3/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



3230.0
3250.0


In [91]:
df_series_far[df_series_far.cp=='c'].iloc[0:1]

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,ert,forward_price,k_over_fp,deltak,p1,p2,p3,e1,e2,e3
0,SPX200221C03225000,2019-12-24 17:44:11,3225.0,56.2,57.0,57.7,-1.349999,-2.345784,643.0,11981,...,1.002919,3226.431401,0.999556,5,0.000028,0.000055,-7.342211e-08,0.000026,-0.000079,-7.669169e-07


In [88]:
df_series_near[df_series_near.cp=='p'].iloc[-1:]

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,...,ert,forward_price,k_over_fp,deltak,p1,p2,p3,e1,e2,e3
119,SPX200117P03220000,2019-12-24 17:42:06,3220.0,29.0,27.7,28.2,0.5,1.754386,74.0,2873.0,...,1.001186,3226.545907,0.997971,5,0.000013,0.000027,-1.644012e-07,5.721933e-07,-0.000002,-2.447783e-09


### sum up p1's p2's and p3's and computer e1, e2, and e3', and create SKEW

In [92]:
def create_skew(df_series):
    e1 = df_series.iloc[0].e1
    e2 = df_series.iloc[0].e2
    e3 = df_series.iloc[0].e3
    ert = df_series.iloc[0].ert
    p1 = ert * -1 * df_series.p1.sum() + e1
    p2 = ert * df_series.p2.sum() + e2
    p3 = ert * df_series.p3.sum() + e3
    S =  (p3 - 3*p1*p2 + 2*p1**3) / (p2 - p1**2)**(3/2)
    SKEW = 100 - 10*S
    return SKEW

In [93]:
Tnear = df_series_near.iloc[0].dte_pct
Tfar = df_series_far.iloc[0].dte_pct
T30 = 30/365
w = (Tfar-T30)/(Tfar-Tnear)
skew_near = create_skew(df_series_near)
skew_far = create_skew(df_series_far)
final_skew = w*skew_near + (1-w) * skew_far
print(final_skew,skew_near,skew_far,w)

148.70917825299028 148.1940089736966 151.19916310290975 0.8285714285714285


### now do multiple days from Oct 2015

In [22]:
home =  pathlib.Path.home()
df_spx_oct_2015= pd.read_csv(f'{home}/downloads/spx_20151001_to_20151030.csv')
df_spx_oct_2015['cp'] = df_spx_oct_2015['type'].apply(lambda v:v[0])
df_spx_oct_2015.tail()


,underlying,underlying_last,exchange,optionroot,optionext,type,expiration,quotedate,strike,last,...,ask,volume,openinterest,impliedvol,delta,gamma,theta,vega,optionalias,cp
241655,SPXW,2084.58,W,SPXW160930P02400000,NaN,put,9/30/16,10/30/15,2400,0.0,...,363.0,0,0,0.1800,-0.7622,0.0009,-54.3014,618.0695,SPXW160930P02400000,p
241656,SPXW,2084.58,W,SPXW160930P02450000,NaN,put,9/30/16,10/30/15,2450,0.0,...,407.9,0,0,0.1879,-0.7853,0.0008,-53.0973,583.2675,SPXW160930P02450000,p
241657,SPXW,2084.58,W,SPXW160930P02500000,NaN,put,9/30/16,10/30/15,2500,0.0,...,455.0,0,0,0.1972,-0.8027,0.0007,-52.7287,554.8251,SPXW160930P02500000,p
241658,SPXW,2084.58,W,SPXW160930P02600000,NaN,put,9/30/16,10/30/15,2600,0.0,...,552.5,0,0,0.2184,-0.8255,0.0006,-53.8173,514.0479,SPXW160930P02600000,p
241659,SPXW,2084.58,W,SPXW160930P02700000,NaN,put,9/30/16,10/30/15,2700,0.0,...,653.8,0,0,0.2421,-0.8379,0.0005,-56.9064,490.2354,SPXW160930P02700000,p


### Get unique quotedate's

In [20]:
quotedates = df_spx_oct_2015.quotedate.unique()
quotedates

array(['10/1/15', '10/2/15', '10/5/15', '10/6/15', '10/7/15', '10/8/15',
       '10/9/15', '10/12/15', '10/13/15', '10/14/15', '10/15/15',
       '10/16/15', '10/19/15', '10/20/15', '10/21/15', '10/22/15',
       '10/23/15', '10/26/15', '10/27/15', '10/28/15', '10/29/15',
       '10/30/15'], dtype=object)

## End